# Некоторое описание датасета (численные параметры)
# MS SubClass: Класс типа недвижимости (например, односемейный дом, многоквартирный дом и т.д.).
# Lot Frontage: Длина участка, обращённого к улице (в футах).
# Lot Area: Площадь участка (в квадратных футах).
# Overall Qual: Общая оценка качества дома (обычно от 1 до 10, где 10 - наилучшее качество).
# Overall Cond: Общая оценка состояния дома (также от 1 до 10).
# Year Built: Год постройки дома.
# Year Remod/Add: Год последнего ремонта или добавления (если дом не был отремонтирован, может быть указано значение 0).
# Mas Vnr Area: Площадь вентилируемого фасада (в квадратных футах).
# BsmtFin SF 1: Площадь, используемая для жилых помещений в подвале (в квадратных футах).
# BsmtFin SF 2: Дополнительная площадь, используемая для жилых помещений в подвале (в квадратных футах).
# Bsmt Unf SF: Площадь неотделанного подвала (в квадратных футах).
# 1st Flr SF: Площадь первого этажа (в квадратных футах).
# 2nd Flr SF: Площадь второго этажа (в квадратных футах).
# Low Qual Fin SF: Площадь, отделанная низкокачественными материалами (в квадратных футах).
# Bsmt Full Bath: Количество полных ванных комнат в подвале.
# Bsmt Half Bath: Количество половинных ванных комнат в подвале.
# Full Bath: Количество полных ванных комнат в доме.
# Half Bath: Количество половинных ванных комнат в доме.
# Bedroom AbvGr: Количество спален на этажах выше уровн земли.
# Kitchen AbvGr: Количество кухонь на этажах выше уровня земли.
# Fireplaces: Количество каминов в доме.
# Garage Yr Blt: Год постройки гаража.
# Garage Cars: Количество автомобилей, которые может вместить гараж.
# Garage Area: Площадь гаража (в квадратных футах).
# Wood Deck SF: Площадь деревянной террасы (в квадратных футах).
# Open Porch SF: Площадь открытой веранды (в квадратных футах).
# Enclosed Porch: Площадь закрытой веранды (в квадратных футах).
# 3Ssn Porch: Площадь веранды, используемой в течение трёх сезонов (в квадратных футах).
# Screen Porch: Площадь веранды с сеткой (в квадратных футах).
# Pool Area: Площадь бассейна (в квадратных футах).
# Misc Val: Разные дополнительные ценности (например, стоимость дополнительных построек или улучшений).
# Mo Sold: Месяц, в который был продан дом (числовое значение от 1 до 12).
# Yr Sold: Год, в который был продан дом.

In [76]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin
from typing import Optional, List

In [27]:
class BaseDataPreprocessor(TransformerMixin):
    def __init__(self, needed_columns: Optional[List[str]]=None, **kwargs):
        """
        :param needed_columns: if not None select these columns from the dataframe
        """

        if needed_columns != None:
            self.needed_cols = needed_columns
        else:
            self.needed_cols = -1
        self.scaler = StandardScaler()

    def fit(self, data, *args):
        self.scaler.fit(data[self.needed_cols if self.needed_cols != -1 else data.columns])
        return self
    def get_params(self, **kwargs):
        return {'needed_columns': self.needed_cols}
    def transform(self, data: pd.DataFrame) -> np.array:
        return self.scaler.transform(data[self.needed_cols if self.needed_cols != -1 else data.columns])


In [139]:
data = pd.read_csv('data/data.csv')
# Избавляемся от выбросов
thrown = pd.concat([data[data['SalePrice'] > 300000], data[data['SalePrice'] < 50000]])
data = data[data['SalePrice'] < 300000]
data = data[data['SalePrice'] > 50000]
# Складываем все веранды
columns_to_sum =  ('Wood Deck SF', 'Open Porch SF', '3Ssn Porch', 'Screen Porch', 'Enclosed Porch')
baths = ['Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath']

data['Bsmt Qual'] = data['Bsmt Qual'].fillna(data['Bsmt Qual'].mode()[0])
data['Bsmt Cond'] = data['Bsmt Cond'].fillna(data['Bsmt Cond'].mode()[0])

dcc_bsmt_obj_to_num = {'TA': 3, 'Gd': 4, 'Ex': 5, 'Fa': 2, 'Po': 1}

def bsmt_c_translator(row):
    return dcc_bsmt_obj_to_num[row]
def bsmt_q_translator(row):
    return dcc_bsmt_obj_to_num[row]

data['Bsmt qual'] = data['Bsmt Qual'].apply(bsmt_q_translator)

data['Bsmt cond'] = data['Bsmt Cond'].apply(bsmt_c_translator)

data['Veranda'] = sum([data[i] for i in columns_to_sum])
data['Baths'] = sum(data[i] for i in baths)

In [108]:
[key for key in data.keys() if data[key].dtype == 'object']

['MS Zoning',
 'Street',
 'Alley',
 'Lot Shape',
 'Land Contour',
 'Utilities',
 'Lot Config',
 'Land Slope',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Mas Vnr Type',
 'Exter Qual',
 'Exter Cond',
 'Foundation',
 'Bsmt Qual',
 'Bsmt Cond',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'BsmtFin Type 2',
 'Heating',
 'Heating QC',
 'Central Air',
 'Electrical',
 'Kitchen Qual',
 'Functional',
 'Fireplace Qu',
 'Garage Type',
 'Garage Finish',
 'Garage Qual',
 'Garage Cond',
 'Paved Drive',
 'Pool QC',
 'Fence',
 'Misc Feature',
 'Sale Type',
 'Sale Condition']

In [114]:
data['Bsmt Qual'].unique()

array(['TA', 'Gd', 'Ex', nan, 'Fa', 'Po'], dtype=object)

In [140]:
def make_ultimate_pipeline():
    chosen_data = ['Lot Area', 'Lot Frontage', 'Overall Cond','Overall Qual', 'Pool Area',
               '1st Flr SF', '2nd Flr SF', 'Veranda', 'Baths', 'Bsmt qual', 'Bsmt cond']
    
    column_tr = ColumnTransformer(
        transformers=[
            ('BasePreprocessor', BaseDataPreprocessor(), chosen_data),
        ]
    )
    imp = SimpleImputer()
    return Pipeline([
        ('columns_transform', column_tr),
        ('Imputer', imp),
        ('KNR', KNeighborsRegressor())
    ])

In [141]:
xtr, xt, ytr, yt = train_test_split(data, data['SalePrice'])

In [142]:
m = make_ultimate_pipeline()
m.fit(xtr, ytr)

Pipeline(steps=[('columns_transform',
                 ColumnTransformer(transformers=[('BasePreprocessor',
                                                  <__main__.BaseDataPreprocessor object at 0x000002331703AD90>,
                                                  ['Lot Area', 'Lot Frontage',
                                                   'Overall Cond',
                                                   'Overall Qual', 'Pool Area',
                                                   '1st Flr SF', '2nd Flr SF',
                                                   'Veranda', 'Baths',
                                                   'Bsmt qual',
                                                   'Bsmt cond'])])),
                ('Imputer', SimpleImputer()), ('KNR', KNeighborsRegressor())])

In [143]:
print('MAE:', mean_absolute_error(yt, m.predict(xt)))
print('MAPE:', mean_absolute_percentage_error(yt, m.predict(xt)))

MAE: 17129.169552238807
MAPE: 0.11464404650842942


In [152]:
def kernel(distances):
    from math import pi
    # distances is an array of size K containing distances of neighbours
    weights = np.exp(-2*distances**2) / np.sqrt(2*pi) # Compute an array of weights however you want
    return distances

params = {
    'KNR__n_neighbors': [i for i in range(7, 20)],
    'KNR__weights': ['distance', 'uniform', kernel]
}
model = GridSearchCV(
    estimator=make_ultimate_pipeline(),
    param_grid=params,
    cv=5,
    scoring='neg_mean_absolute_error',
)
model.fit(data, data['SalePrice'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columns_transform',
                                        ColumnTransformer(transformers=[('BasePreprocessor',
                                                                         <__main__.BaseDataPreprocessor object at 0x000002331AAC99D0>,
                                                                         ['Lot '
                                                                          'Area',
                                                                          'Lot '
                                                                          'Frontage',
                                                                          'Overall '
                                                                          'Cond',
                                                                          'Overall '
                                                                          'Qual',
                                                                          'Pool '
                                                                          'Area',
                                                                          '1st '
                                                                          'Flr '
                                                                          'SF',
                                                                          '2nd '
                                                                          'Flr '
                                                                          'SF',
                                                                          'Veranda',
                                                                          'Baths',
                                                                          'Bsmt '
                                                                          'qual',
                                                                          'Bsmt '
                                                                          'cond'])])),
                                       ('Imputer', SimpleImputer()),
                                       ('KNR', KNeighborsRegressor())]),
             param_grid={'KNR__n_neighbors': [7, 8, 9, 10, 11, 12, 13, 14, 15,
                                              16, 17, 18, 19],
                         'KNR__weights': ['distance', 'uniform',
                                          <function kernel at 0x000002331A60F6A0>]},
             scoring='neg_mean_absolute_error')

In [153]:
model.best_score_

-17343.982434657904

Общие заметки по данным:
1) параметров очень много --- обязательно проводить конкретный выбор их них.
2) некоторые из фич семантически близки: количество различных ванн, количество веранд разного типа и т.д. 
3) есть параметры наиболее хорошо выражающие взаимосвязь с ценой (очевидно, это параметры overall cond и qual)

общее заключение такое, что данные слишком сложно устроены, чтобы пытаться предсказать их с помощью KNR.